#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [45]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# loading data and data ingestion project
lodaer = TextLoader('speech.txt')
documents = lodaer.load()
# splitting data into chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)
docs

# now will convert into embedding and storing into FAISS db
embeddings = OllamaEmbeddings(model="nomic-embed-text")
db = FAISS.from_documents(docs, embeddings)

# Now we will query some text using similarity search

query = "How does the speaker describe the desired outcome of the war?"
query_results = db.similarity_search(query)

query_results[0].page_content

# Read entire results
# for i in query_results:
#     print(i.page_content)

# db search as retriver

retriever = db.as_retriever()
retriever_results = retriever.invoke(query)
retriever_results[0].page_content

# db similarity search and score

docs_score = db.similarity_search_with_score(query)
docs_score

# embed query
embedding_vector = embeddings.embed_query(query)
embedding_vector



[-0.2991303503513336,
 0.2880570590496063,
 -3.5440778732299805,
 -0.617120087146759,
 2.103498935699463,
 1.4999171495437622,
 -0.9283971190452576,
 -0.03791424259543419,
 0.6267483234405518,
 -0.6312190294265747,
 -0.17841942608356476,
 0.8109188079833984,
 0.8038966059684753,
 1.4821908473968506,
 2.0426888465881348,
 -0.8410398960113525,
 0.4809218645095825,
 -1.1228317022323608,
 -0.839410126209259,
 0.8148530125617981,
 -1.1434831619262695,
 -0.39906761050224304,
 0.12177904695272446,
 0.2861455976963043,
 1.6115020513534546,
 0.5302199721336365,
 -0.38199177384376526,
 0.7436512112617493,
 -1.0927318334579468,
 -0.35064318776130676,
 1.1465094089508057,
 -0.2853994369506836,
 -0.265272319316864,
 0.28941211104393005,
 -1.9662230014801025,
 -1.6483412981033325,
 0.3919660151004791,
 1.0429160594940186,
 0.514140784740448,
 -1.4721626043319702,
 0.12473313510417938,
 -0.28971630334854126,
 0.10435245931148529,
 -1.5336564779281616,
 1.3150662183761597,
 -0.3696536123752594,
 0.011

In [35]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [11]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [12]:
# embeddings=OllamaEmbeddings()
embeddings=(
    OllamaEmbeddings(model="nomic-embed-text")  ##by default it ues llama2
)
db=FAISS.from_documents(docs,embeddings)
db

In [13]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [14]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [15]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='b2673e0c-712d-476c-bfdc-ad0eaf9b2860', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(314.83405)),
 (Document(id='97083940-64c8-4d96-9ca7-d022b176237d', metadata={'source': 'spe

In [16]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-0.2991303503513336,
 0.2880570590496063,
 -3.5440778732299805,
 -0.617120087146759,
 2.103498935699463,
 1.4999171495437622,
 -0.9283971190452576,
 -0.03791424259543419,
 0.6267483234405518,
 -0.6312190294265747,
 -0.17841942608356476,
 0.8109188079833984,
 0.8038966059684753,
 1.4821908473968506,
 2.0426888465881348,
 -0.8410398960113525,
 0.4809218645095825,
 -1.1228317022323608,
 -0.839410126209259,
 0.8148530125617981,
 -1.1434831619262695,
 -0.39906761050224304,
 0.12177904695272446,
 0.2861455976963043,
 1.6115020513534546,
 0.5302199721336365,
 -0.38199177384376526,
 0.7436512112617493,
 -1.0927318334579468,
 -0.35064318776130676,
 1.1465094089508057,
 -0.2853994369506836,
 -0.265272319316864,
 0.28941211104393005,
 -1.9662230014801025,
 -1.6483412981033325,
 0.3919660151004791,
 1.0429160594940186,
 0.514140784740448,
 -1.4721626043319702,
 0.12473313510417938,
 -0.28971630334854126,
 0.10435245931148529,
 -1.5336564779281616,
 1.3150662183761597,
 -0.3696536123752594,
 0.011

In [13]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.', metadata={'source': 'speech.txt'}),
 Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the prid

In [14]:
### Saving And Loading
db.save_local("faiss_index")

In [16]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [17]:
docs

[Document(page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.', metadata={'source': 'speech.txt'}),
 Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the prid